In [1]:
from tensorflow.keras import datasets
from tensorflow.keras.models import Model
from keras.applications.resnet50 import ResNet50
import cv2 as cv
import numpy as np
import tensorflow as tf
import tensorflow_addons as tfa
import matplotlib.pyplot as plt

Couldn't import dot_parser, loading of dot files will not be possible.


In [2]:
#train_images = tf.pad(train_images, [[0, 0], [2,2], [2,2], [0,0]])
#test_images = tf.pad(test_images, [[0, 0], [2,2], [2,2], [0,0]])
def padImage(image, pixels=5):
    bottom = image[-pixels:]
    top = image[:pixels]

    img = np.insert(image, 0, bottom, 0)
    img = np.insert(img, len(img), top, 0)
    #img = np.insert(img, [0], [0] * pixels, 1)
    #img = np.insert(img, [-1], [0] * pixels, 1)
    return img

In [5]:
# Load the MNIST dataset
(train_images, train_labels), (test_images, test_labels) = datasets.mnist.load_data()
# Normalize pixel values to be between 0 and 1
train_images, test_images = (train_images / 255.0).astype(np.float32), test_images.astype(np.float32) / 255.0

#train_images = train_images.reshape(len(train_images), 28, 28, 1)
#test_images = test_images.reshape(len(test_images), 28, 28, 1)

In [21]:
### THE PAD HAS TO BE DONE IN THE
### POLAR SPACE

# 20 is the ceiling of (14 * sqrt(2))
X_train_polar = [cv.linearPolar(x, tuple(np.array(x.shape)/2), 20, cv.WARP_FILL_OUTLIERS) for x in train_images]
X_train_polar = [padImage(x, pixels=2) for x in X_train_polar]
X_train_polar = np.array(X_train_polar)[...,None]
X_train_polar = tf.pad(X_train_polar, [[0, 0], [0,0], [2,2], [0,0]]) # pad 2 pixels each side to make 32x32

X_test_polar = [cv.linearPolar(x, tuple(np.array(x.shape)/2), 20, cv.WARP_FILL_OUTLIERS) for x in test_images]
X_test_polar = [padImage(x, pixels=5) for x in X_test_polar]
X_test_polar = np.array(X_test_polar)[...,None]
X_test_polar = tf.pad(X_test_polar, [[0, 0], [0,0], [2,2], [0,0]]) # pad 2 pixels each side to make 32x32

# Rotate test set
X_test_r_polar = [tfa.image.rotate(x, np.random.uniform(-np.pi/2., np.pi/2.)).numpy() for x in test_images]
X_test_r_polar = [cv.linearPolar(x, tuple(np.array(x.shape)/2), 20, cv.WARP_FILL_OUTLIERS) for x in X_test_r_polar]
X_test_r_polar = [padImage(x, pixels=5) for x in X_test_r_polar]
X_test_r_polar = np.array(X_test_r_polar)[...,None]
X_test_r_polar = tf.pad(X_test_r_polar, [[0, 0], [0,0], [2,2], [0,0]]) # pad 2 pixels each side to make 32x32

In [24]:
input = tf.keras.layers.Input(shape=(32,32,1))
base = ResNet50(include_top=False, weights=None, input_shape=(32,32,1))
base = base(input)


In [25]:
out = tf.keras.layers.Dense(10, activation='softmax')(base)

In [26]:
model = tf.keras.models.Model(input, out)

In [27]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [28]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_11 (InputLayer)        [(None, 32, 32, 1)]       0         
_________________________________________________________________
resnet50 (Functional)        (None, 1, 1, 2048)        23581440  
_________________________________________________________________
dense (Dense)                (None, 1, 1, 10)          20490     
Total params: 23,601,930
Trainable params: 23,548,810
Non-trainable params: 53,120
_________________________________________________________________


In [31]:
model.fit(X_train_polar, train_labels, epochs=10, batch_size=128)

Epoch 1/10
  4/469 [..............................] - ETA: 31:32 - loss: 3.7165 - accuracy: 0.1006

KeyboardInterrupt: 